In [1]:
#load "Paket.fsx"
Paket.Package
  [ "QBitNinja.Client -Version 1.0.3.46"
    "NBitcoin -Version 4.0.0.48"
    "FSharp.Data -Version 2.4.6"
    "Newtonsoft.Json -Version 10.0.3"    
  ]    
#load "Paket.Generated.Refs.fsx"

open System
open NBitcoin
open NBitcoin.Crypto   
open NBitcoin.OpenAsset 
open NBitcoin.Stealth 
open QBitNinja.Client 
open QBitNinja.Client.Models 
open System.Collections.Generic 
open System.Linq
open System.Collections   

open FSharp.Data
open FSharp.Data.JsonExtensions

type secretJson = JsonProvider<"""{
  "secret": ""
}""">
      
let sj = secretJson.GetSample()

In [6]:
let alice = secretJson.Load(@"/home/nbuser/library/secret.json").JsonValue?secret.AsString()  
let bob = secretJson.Load(@"/home/nbuser/library/secret_dest.json").JsonValue?secret.AsString()  
let zorro = secretJson.Load(@"/home/nbuser/library/secret_zorro.json").JsonValue?secret.AsString()  
  
let happy = secretJson.Load(@"/home/nbuser/library/secret_zorro.json").JsonValue?secret.AsString() 

let alice_private  = Key.Parse(alice, Network.TestNet) 
let bob_private  = Key.Parse(bob, Network.TestNet) 
let zorro_private  = Key.Parse(zorro, Network.TestNet) 
let happy_private  = Key.Parse(zorro, Network.TestNet)   

In [7]:
let ZorroComp:Script = 
                PayToMultiSigTemplate
                            .Instance
                            .GenerateScriptPubKey(2,  [| alice_private.PubKey; bob_private.PubKey; zorro_private.PubKey |])

In [8]:
printfn "%A" ZorroComp

2 0364dfe2fa8c584d9af693fca129f9c031e4108be2b6d31ac3f6846ef1ba7ecdc1 0230621e0bd25d7207cacd09368b62b4107bca1a416f12d86e4f159e718e9d7750 02016e47a7061e49d9e0b13662e8c6efa0c6628051c11717ebb229f01231a0c5f0 3 OP_CHECKMULTISIG


In [9]:
let client:QBitNinjaClient  = new QBitNinjaClient(Network.TestNet)

let transactionId = uint256.Parse("d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b")

// Query the transaction
let transactionResponse:GetTransactionResponse  = client.GetTransaction(transactionId).Result

let transaction:NBitcoin.Transaction  = transactionResponse.Transaction

let receivedCoins = transactionResponse.ReceivedCoins;
let mutable myCoins:Coin = null 
for coin in receivedCoins do
    if (coin.TxOut.ScriptPubKey =  alice_private.ScriptPubKey) then
        myCoins<- coin:?>Coin
        
printfn "%A" myCoins.Amount

0.53990000


In [10]:
let address = ZorroComp.GetScriptAddress(Network.TestNet)

In [11]:
printfn "%A" address

2MsiuDAu1e7mjHrMKGTXGBUXQwG1qjJVqMe


In [14]:


let mutable txBuilder = new TransactionBuilder() 
let mutable tx:Transaction = txBuilder
                                .AddCoins(myCoins) 
                                .AddKeys(alice_private, bob_private)
                                .Send(happy_private, new Money(0.1m, MoneyUnit.BTC))
                                .SetChange(address)
                                .SendFees(new Money(0.001m, MoneyUnit.BTC))
                                .BuildTransaction(true)


In [15]:
printfn "%A" tx



{
  "hash": "5d66576941d2c7ae21c1bfd47dd8f7399ec64fd564080a25a89e0e0239dff8f0",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 2,
  "lock_time": 0,
  "size": 223,
  "in": [
    {
      "prev_out": {
        "hash": "d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b",
        "n": 0
      },
      "scriptSig": "30440220354e69cd570b57056df65037f5999c780953e971e5b7a8f330b9d8efe86699aa022007e4694ff401c9b1b49f2dfff016a2fa58186ad98f51bff6cc14e1d71cf71f6801 0364dfe2fa8c584d9af693fca129f9c031e4108be2b6d31ac3f6846ef1ba7ecdc1"
    }
  ],
  "out": [
    {
      "value": "0.43890000",
      "scriptPubKey": "OP_HASH160 053bf8d5af547a231ac93e517c43f91c87e2c8c5 OP_EQUAL"
    },
    {
      "value": "0.10000000",
      "scriptPubKey": "OP_DUP OP_HASH160 898a3acffbef7c880ee82c197d79e96f3025982d OP_EQUALVERIFY OP_CHECKSIG"
    }
  ]
}
